# Initialization

In [10]:
import mlxtend.frequent_patterns
import numpy as np
import pandas as pd
import polars as pl
import whoosh_utils
from collections import Counter
from pathlib import Path
from tqdm.notebook import tqdm

is_kaggle = Path("/kaggle").is_dir()

if is_kaggle:
    competition_data_directory = Path("/kaggle/input/uspto-explainable-ai")
    validation_data_directory = Path("/kaggle/input/uspto-explainable-ai-validation-index")
    output_directory = Path("/kaggle/working")
else:
    competition_data_directory = Path("/media/jasper/Jasper-T7/uspto-explainable-ai/competition-data")
    validation_data_directory = Path("/media/jasper/Jasper-T7/uspto-explainable-ai/validation-data")
    output_directory = Path("/media/jasper/Jasper-T7/uspto-explainable-ai/original-submission")

max_query_tokens = 50

print(f"{is_kaggle=}")
print(f"{competition_data_directory=}")
print(f"{validation_data_directory=}")
print(f"{output_directory=}")
print(f"{max_query_tokens=}")

is_kaggle=False
competition_data_directory=PosixPath('/media/jasper/Jasper-T7/uspto-explainable-ai/competition-data')
validation_data_directory=PosixPath('/media/jasper/Jasper-T7/uspto-explainable-ai/validation-data')
output_directory=PosixPath('/media/jasper/Jasper-T7/uspto-explainable-ai/original-submission')
max_query_tokens=50


In [2]:
df_test = pl.read_csv(competition_data_directory / "test.csv")
df_test

publication_number,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22,target_23,target_24,target_25,target_26,target_27,target_28,target_29,target_30,target_31,target_32,target_33,target_34,target_35,target_36,target_37,target_38,target_39,target_40,target_41,target_42,target_43,target_44,target_45,target_46,target_47,target_48,target_49
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""US-2017082634-A1""","""US-2020225242-A1""","""US-2010137151-A1""","""US-10607721-B2""","""US-6747273-B2""","""US-6940065-B2""","""US-7684934-B2""","""US-2019018928-A1""","""US-9040903-B2""","""US-7847245-B2""","""US-8501421-B2""","""US-9110076-B2""","""US-10617717-B2""","""US-8741556-B2""","""US-9312110-B2""","""US-7865312-B2""","""US-11428696-B2""","""US-11646185-B2""","""US-7603240-B2""","""US-9879304-B2""","""US-9354236-B2""","""US-2022365028-A1""","""US-2014051105-A1""","""US-2021239708-A1""","""US-10825672-B2""","""US-2022397578-A1""","""US-9746477-B2""","""US-2022221467-A1""","""US-2016139140-A1""","""US-2019353660-A1""","""US-9110074-B2""","""US-8604692-B2""","""US-2017168055-A1""","""US-7499807-B1""","""US-2003008327-A1""","""US-2003166007-A1""","""US-10598642-B2""","""US-10401337-B2""","""US-2005221500-A1""","""US-2017023547-A1""","""US-2003157575-A1""","""US-10197576-B2""","""US-11011359-B2""","""US-7020559-B1""","""US-8642349-B1""","""US-2009253156-A1""","""US-7783429-B2""","""US-2003060983-A1""","""US-10914740-B2""","""US-7858389-B2""","""US-9747412-B2"""
"""US-2017180470-A1""","""US-2017171349-A1""","""US-2017171301-A1""","""US-2017155712-A1""","""US-2017163642-A1""","""US-2017155740-A1""","""US-2017180760-A1""","""US-2017155927-A1""","""US-2014289307-A1""","""US-9550119-B2""","""US-2017163478-A1""","""US-9280384-B2""","""US-2017155928-A1""","""US-2017163479-A1""","""US-10798147-B2""","""US-2017171150-A1""","""US-11582325-B2""","""US-11057489-B2""","""US-2017150214-A1""","""US-10182126-B2""","""US-9456024-B1""","""US-2017155695-A1""","""US-2017163706-A1""","""US-2021234919-A1""","""US-2014073370-A1""","""US-10075387-B1""","""US-2012005274-A1""","""US-8180857-B2""","""US-2017171566-A1""","""US-11432171-B2""","""US-2018227220-A1""","""US-10231269-B2""","""US-2017163761-A1""","""US-2017180468-A1""","""US-2022263759-A1""","""US-11582153-B2""","""US-2019245912-A1""","""US-2021389960-A1""","""US-2017171147-A1""","""US-10958580-B2""","""US-10897506-B2""","""US-11586626-B1""","""US-10945130-B2""","""US-2023041645-A1""","""US-2014310385-A1""","""US-9749174-B1""","""US-2014344473-A1""","""US-2017142196-A1""","""US-9276860-B2""","""US-2017149934-A1""","""US-2017153909-A1"""
"""US-2018029544-A1""","""US-9597956-B2""","""US-2015197150-A1""","""US-2015034146-A1""","""US-2012132245-A1""","""US-2014137939-A1""","""US-2011073163-A1""","""US-10916749-B2""","""US-9350288-B2""","""US-2020105953-A1""","""US-8875828-B2""","""US-9016765-B1""","""US-2017104446-A1""","""US-2014174506-A1""","""US-5741370-A""","""US-2012240982-A1""","""US-2011226335-A1""","""US-2011146761-A1""","""US-2011315191-A1""","""US-2021193855-A1""","""US-11394071-B2""","""US-10211355-B2""","""US-2012080074-A1""","""US-5762720-A""","""US-2015194924-A1""","""US-2019319145-A1""","""US-2012315528-A1""","""US-2012103401-A1""","""US-2022367940-A1""","""US-8952236-B2""","""US-11431044-B2""","""US-2016043257-A1""","""US-6631943-B2""","""US-2013000689-A1""","""US-2011155221-A1""","""US-2023011595-A1""","""US-9722116-B2""","""US-2013118556-A1""","""US-2014345833-A1""","""US-2019035962-A1""","""US-9278617-B2""","""US-8748737-B2""","""US-4717790-A""","""US-2022320600-A1""","""US-10784390-B2""","""US-11152660-B2""","""US-2022407163-A1""","""US-8056664-B2""","""US-2022140428-A1

# Query Builder

In [3]:
df_patent_metadata = pl.read_parquet(competition_data_directory / "patent_metadata.parquet")
df_patent_metadata = df_patent_metadata.with_columns(family_id=df_patent_metadata["family_id"].cast(pl.Int32))
df_patent_metadata

publication_number,publication_date,filing_date,family_id,cpc_codes
str,datetime[μs],datetime[μs],i32,list[str]
"""US-1-A""",1836-07-13 00:00:00,null,2060279,"[""B61C11/04""]"
"""US-1-P""",1931-08-18 00:00:00,1930-08-06 00:00:00,49893162,"[""A01H5/02"", ""A01H6/749""]"
"""US-10-A""",1836-08-10 00:00:00,null,2060288,"[""B27J1/00""]"
"""US-1000-A""",1838-11-03 00:00:00,null,2061286,"[""B60G11/04"", ""Y10S507/905""]"
"""US-10000-A""",1853-09-06 00:00:00,null,2070320,"[""F04D29/283""]"
…,…,…,…,…
"""US-T999003-I4""",1980-10-07 00:00:00,1980-04-25 00:00:00,3067331,"[""G03C8/10""]"
"""US-X1-I1""",1790-07-31 00:00:00,1790-07-31 00:00:00,40303722,"[""C01D3/08""]"
"""US-X7668-I1""",1833-07-22 00:00:00,null,53773065,[]


In [4]:
publication_number_index = {v: i for i, v in enumerate(df_patent_metadata["publication_number"].to_list())}
publication_number_index["US-1-A"], publication_number_index["US-2017082634-A1"]

(0, 3738871)

In [5]:
term_counter = Counter()
term_counter_patents = 0

stopwords_pattern = "|".join(f"\\b{word}\\b" for word in [
    "an", "are", "by", "for", "if", "into", "is", "no", "not", "of", "on", "such",
    "that", "the", "their", "then", "there", "these", "they", "this", "to", "was", "will",
])

def add_terms_to_counter(terms: pl.Series, prefix: str) -> None:
    for term, count in terms.value_counts(parallel=True).rows():
        if term is not None and term != "":
            term_counter[f"{prefix}:{term}"] += count

def add_texts_to_counter(texts: pl.Series, prefix: str) -> None:
    terms = texts \
        .str.to_lowercase() \
        .str.replace_all(fr"\d+|{stopwords_pattern}", "") \
        .str.split(" ") \
        .list.unique() \
        .list.explode()

    add_terms_to_counter(terms, prefix)

add_terms_to_counter(df_patent_metadata["cpc_codes"].list.explode(), "cpc")

for patent_data_file in tqdm(list((competition_data_directory / "patent_data").iterdir())):
    df = pl.read_parquet(patent_data_file, columns=["title"])

    add_texts_to_counter(df["title"], "ti")
    term_counter_patents += df.shape[0]

term_selectivity = {term: count / term_counter_patents for term, count in term_counter.items()}

most_common = term_counter.most_common(5)
most_common, {term: term_selectivity[term] for term, _ in most_common}

  0%|          | 0/2251 [00:00<?, ?it/s]

([('ti:and', 4053669),
  ('ti:method', 2345794),
  ('ti:a', 1444966),
  ('ti:device', 1260129),
  ('ti:apparatus', 1239351)],
 {'ti:and': 0.3046117506175767,
  'ti:method': 0.1762739920127193,
  'ti:a': 0.108581540042583,
  'ti:device': 0.09469201868578228,
  'ti:apparatus': 0.09313066205939467})

In [6]:
whoosh_text_analyzer = whoosh_utils._define_uspto_whoosh_schema()["ti"].analyzer

def extract_text_tokens(text: str) -> list[str]:
    return [token.text for token in whoosh_text_analyzer(text)]

def get_terms_by_publication_number(publication_number: str) -> set[str]:
    metadata_idx = publication_number_index[publication_number]

    publication_date = df_patent_metadata["publication_date"][metadata_idx]
    if publication_date is not None:
        year, month = publication_date.year, publication_date.month
    else:
        year, month = "nan", "nan"

    df_patent_data = pl.scan_parquet(competition_data_directory / "patent_data" / f"{year}_{month}.parquet") \
        .filter(pl.col("publication_number") == publication_number) \
        .limit(1) \
        .select(pl.col(["title"])) \
        .collect(streaming=True)

    cpc_codes = df_patent_metadata["cpc_codes"][metadata_idx]
    cpc_terms = [f"cpc:{code}" for code in cpc_codes]

    title = df_patent_data["title"][0]
    title_terms = [f"ti:{token}" for token in extract_text_tokens(title)]

    return set(cpc_terms + title_terms)

get_terms_by_publication_number("US-2017082634-A1")

{'cpc:C12Q1/485',
 'cpc:G01N2570/00',
 'cpc:G01N33/6845',
 'cpc:G01N33/6848',
 'cpc:G06F19/20',
 'cpc:G16B25/00',
 'cpc:G16B25/10',
 'cpc:G16B5/00',
 'ti:and',
 'ti:multiplexed',
 'ti:phosphoproteomics',
 'ti:proteomics'}

In [7]:
def serialize_term_group(group: list[str]) -> str:
    if len(group) == 1:
        return group[0]
    else:
        return "(" + " ".join(group) + ")"

def serialize_term_groups(or_groups: list[list[str]], xor_groups: list[list[str]]) -> str:
    or_subquery = " OR ".join(map(serialize_term_group, or_groups))
    xor_subquery = " XOR ".join(map(serialize_term_group, xor_groups))

    if len(or_groups) > 0 and len(xor_groups) > 0:
        return f"({or_subquery}) XOR {xor_subquery}"
    elif len(or_groups) > 0:
        return or_subquery
    elif len(xor_groups) > 0:
        return xor_subquery
    else:
        return ""

serialize_term_groups([["a", "b"], ["c", "d"]], [["e", "f"], ["g", "h", "i"]])

'((a b) OR (c d)) XOR (e f) XOR (g h i)'

In [8]:
def get_itemset_selectivity(itemset: frozenset) -> float:
    selectivity = 1

    for term in itemset:
        selectivity *= term_selectivity.get(term, 1)

    return selectivity

get_itemset_selectivity(frozenset(["ti:and"])), get_itemset_selectivity(frozenset(["ti:device"])), get_itemset_selectivity(frozenset(["ti:and", "ti:device"]))

(0.3046117506175767, 0.09469201868578228, 0.028844301581388426)

In [11]:
def build_query(row: tuple[str, ...]) -> str:
    targets = row[1:]

    terms_by_target = list(map(get_terms_by_publication_number, targets))
    unique_terms = set().union(*terms_by_target)

    if len(unique_terms) == 0:
        return "ti:device"

    df_mlxtend = {term: [] for term in unique_terms}
    for target_terms in terms_by_target:
        for term in unique_terms:
            df_mlxtend[term].append(term in target_terms)

    df_mlxtend = pd.DataFrame(df_mlxtend)
    df_mlxtend = mlxtend.frequent_patterns.fpgrowth(df_mlxtend, min_support=1 / len(targets) * 1, max_len=2, use_colnames=True)
    df_mlxtend["selectivity"] = np.round(df_mlxtend["itemsets"].apply(get_itemset_selectivity), 8)
    df_mlxtend["score"] = df_mlxtend["support"] * df_mlxtend["selectivity"]
    df_mlxtend = df_mlxtend.sort_values(["score", "support"], ascending=[True, False])

    groups: list[tuple[list[str], list[int]]] = []
    target_coverage = [0] * len(targets)

    for row in df_mlxtend.itertuples():
        group = row.itemsets

        new_query = serialize_term_groups([list(group)], [list(tup[0]) for tup in groups])
        if whoosh_utils.count_query_tokens(new_query) > max_query_tokens:
            break

        covered_targets = []
        covers_new_target = False

        for i, target_terms in enumerate(terms_by_target):
            if len(group & target_terms) == len(group):
                covered_targets.append(i)
                covers_new_target = covers_new_target or target_coverage[i] == 0

        if not covers_new_target:
            continue

        groups.append((list(group), covered_targets))
        for target in covered_targets:
            target_coverage[target] += 1

    or_groups: list[list[str]] = []
    xor_groups: list[list[str]] = []

    for group, covered_targets in groups:
        if all(target_coverage[target] == 1 for target in covered_targets):
            xor_groups.append(group)
        else:
            or_groups.append(group)

    if len(or_groups) == 0 and len(xor_groups) == 0:
        return list(unique_terms)[0]

    return serialize_term_groups(or_groups, xor_groups)

def build_query_safe(row: tuple[str, ...]) -> str:
    try:
        return build_query(row)
    except:
        return "ti:device"

build_query(df_test.row(0))

'((cpc:G16B40/10 cpc:H01J49/0036) OR (ti:proteomics cpc:G01N33/6848) OR (cpc:G01N2458/15 cpc:G01N33/6848) OR (cpc:G01N33/6848 cpc:H01J49/00) OR (ti:proteomics ti:multiplexed) OR (cpc:H01J49/0027 cpc:G16B40/10) OR (cpc:H01J49/0027 cpc:H01J49/0036) OR (cpc:H01J49/0027 cpc:H01J49/004) OR (cpc:G16B40/10 cpc:G06F19/24) OR (cpc:G16C20/70 cpc:G16C20/20) OR (cpc:G01N33/6848 cpc:G16C20/90) OR (cpc:G01N2560/00 cpc:G01N2570/00)) XOR (cpc:G01N33/6842 cpc:G01N33/6851) XOR (ti:kinases ti:phosphatases) XOR (cpc:H01J49/0031 cpc:G01N2560/00) XOR (cpc:G16B30/00 ti:proteomic) XOR (cpc:G01N33/6818 cpc:G16B30/00)'

# Validation

In [12]:
if not is_kaggle:
    validation_query_parser = whoosh_utils.get_query_parser()
    validation_index = whoosh_utils.load_index(str(validation_data_directory / "validation" / "validation_index"))
    validation_searcher = whoosh_utils.get_searcher(validation_index)

    def execute_validation_query(query: str) -> list[str]:
        return whoosh_utils.execute_query(query, validation_query_parser, validation_searcher)

    # https://patents.google.com/patent/US10098246B1
    print(execute_validation_query("ti:display AND ti:system AND ti:and AND ti:method AND ti:use"))

['US-2003210238-A1', 'US-10098246-B1', 'US-7716354-B2']


In [13]:
if not is_kaggle:
    df_validation = pl.read_csv(validation_data_directory / "neighbors_small.csv")
else:
    df_validation = None

df_validation

publication_number,neighbor_0,neighbor_1,neighbor_2,neighbor_3,neighbor_4,neighbor_5,neighbor_6,neighbor_7,neighbor_8,neighbor_9,neighbor_10,neighbor_11,neighbor_12,neighbor_13,neighbor_14,neighbor_15,neighbor_16,neighbor_17,neighbor_18,neighbor_19,neighbor_20,neighbor_21,neighbor_22,neighbor_23,neighbor_24,neighbor_25,neighbor_26,neighbor_27,neighbor_28,neighbor_29,neighbor_30,neighbor_31,neighbor_32,neighbor_33,neighbor_34,neighbor_35,neighbor_36,neighbor_37,neighbor_38,neighbor_39,neighbor_40,neighbor_41,neighbor_42,neighbor_43,neighbor_44,neighbor_45,neighbor_46,neighbor_47,neighbor_48,neighbor_49
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""US-10004275-B2""","""US-10238147-B2""","""US-9814272-B2""","""US-2016135504-A1""","""US-9999253-B2""","""US-10661032-B2""","""US-10582727-B2""","""US-10004274-B2""","""US-2021235765-A1""","""US-9993028-B2""","""US-10321715-B2""","""US-10064433-B2""","""US-9861131-B2""","""US-10721966-B2""","""US-10660367-B2""","""US-10893706-B2""","""US-10085484-B2""","""US-10575559-B2""","""US-10125004-B2""","""US-10285447-B2""","""US-9861136-B2""","""US-9894937-B2""","""US-10377622-B2""","""US-9968135-B2""","""US-2016338413-A1""","""US-9492976-B2""","""US-10791759-B2""","""US-10064431-B2""","""US-10196188-B2""","""US-9801417-B2""","""US-10070668-B2""","""US-10292428-B2""","""US-9848650-B2""","""US-10524504-B2""","""US-9220303-B2""","""US-9193229-B2""","""US-9498044-B2""","""US-10869507-B2""","""US-2015377354-A1""","""US-9901122-B2""","""US-9205386-B2""","""US-11077262-B2""","""US-11147314-B2""","""US-9894935-B2""","""US-11103008-B2""","""US-10172393-B2""","""US-9814269-B2""","""US-2592808-A""","""US-2017224017-A1""","""US-9078475-B2""","""US-2015118071-A1"""
"""US-10004377-B2""","""US-9326658-B2""","""US-2016302642-A1""","""US-8967165-B2""","""US-9339166-B2""","""US-2011232690-A1""","""US-9693670-B2""","""US-7942156-B2""","""US-2017354307-A1""","""US-2006237050-A1""","""US-8104489-B2""","""US-5097855-A""","""US-10022034-B2""","""US-2015107632-A1""","""US-2022304545-A1""","""US-9314145-B2""","""US-7635003-B2""","""US-10827903-B2""","""US-4150679-A""","""US-10512386-B2""","""US-11612298-B2""","""US-2015129511-A1""","""US-3989054-A""","""US-3090391-A""","""US-3575185-A""","""US-1223380-A""","""US-2018192851-A1""","""US-9968236-B2""","""US-2018317738-A1""","""US-9010344-B2""","""US-9918610-B2""","""US-11166615-B2""","""US-2014158168-A1""","""US-2009007942-A1""","""US-2014238447-A1""","""US-2014202502-A1""","""US-2015096599-A1""","""US-2020214529-A1""","""US-5711325-A""","""US-5849101-A""","""US-3217884-A""","""US-2013104935-A1""","""US-9492052-B2""","""US-4346723-A""","""US-11160433-B2""","""US-3502090-A""","""US-2015129512-A1""","""US-9839339-B2""","""US-10653291-B2""","""US-8834647-B2""","""US-9649006-B2"""
"""US-10006748-B2""","""US-11192659-B2""","""US-9976636-B2""","""US-10337500-B2""","""US-8581119-B2""","""US-2015093927-A1""","""US-9254794-B2""","""US-2022357136-A1""","""US-2879088-A""","""US-8893593-B2""","""US-2866244-A""","""US-2944864-A""","""US-9458889-B2""","""US-9592917-B2""","""US-9453525-B2""","""US-10526823-B2""","""US-9470023-B2""","""US-10247285-B2""","""US-9939237-B2""","""US-11300150-B2""","""US-11041551-B2""","""US-2016305165-A1""","""US-9368911-B2""","""US-5397244-A""","""US-3362237-A""","""US-9583877-B1""","""US-10595435-B2""","""US-8016327-B2""","""US-10024376-B2""","""US-9573694-B2""","""US-2015155668-A1""","""US-10435929-B2""","""US-9806478-B2""","""US-3197552-A""","""US-7148438-B2""","""US-10081434-B2""","""US-9112307-B2""","""US-6935805-B2""","""US-2021300578-A1""","""US-9091506-B2""","""US-4232978-A""","""US-10138927-B2""","""US-10218122-B1""","""US-7896570-B2""","""US-2017198744-A1""","""US-4393729-A""","""US-10465666-B2""","""US-8250980-B1""","""US-2015336675-A1""","""US-9593708-B2""","""US-2022136557-A1"""
"""US-

In [14]:
if not is_kaggle:
    scores = []

    progress = tqdm(total=df_validation.shape[0])
    for row in df_validation.rows():
        query = build_query(row)
        result = set(execute_validation_query(query))

        hits = sum(target in result for target in row[1:])
        score = hits / (len(row) - 1)

        scores.append(score)

        progress.set_postfix({"score": f"{np.mean(scores):,.3f}±{np.std(scores):,.3f}", "last_score": f"{score:,.3f}"})
        progress.update()

  0%|          | 0/3998 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Submission

In [15]:
submission = pl.DataFrame({
    "publication_number": df_test["publication_number"],
    "query": df_test.map_rows(build_query_safe),
})

submission

publication_number,query
str,str
"""US-2017082634-A1""","""((cpc:G16B40/10 cpc:H01J49/003…"
"""US-2017180470-A1""","""(cpc:H04N21/632 cpc:H04N21/234…"
"""US-2018029544-A1""","""((cpc:B60K2016/003 cpc:B60K16/…"
"""US-2022408153-A1""","""((cpc:H04N21/8549 cpc:H04N21/4…"
"""US-2268569-A""","""((cpc:E02F3/3486 cpc:E02F9/022…"
"""US-3371854-A""","""((cpc:H01J41/16 ti:getter) OR …"
"""US-3589189-A""","""(ti:meters cpc:G01F3/223) XOR …"
"""US-3881203-A""","""(cpc:B42F9/00 cpc:Y10T24/206) …"
"""US-4845770-A""","""(cpc:H04N1/107 cpc:G06V30/142)…"


In [16]:
output_directory.mkdir(parents=True, exist_ok=True)
submission.write_csv(output_directory / "submission.csv")